In [1]:
import geopandas as gpd
from shapely import LineString, Polygon, Point
import shapely
from shapely import ops
import matplotlib.pyplot as plt
from CPP_Planner import CPP_Algorithms, CPP_Planner_Kit
from shapely import affinity
from skimage.measure import approximate_polygon
import math
import warnings

# warnings.filterwarnings('ignore')
all_land = gpd.read_file('test_Load_Shp/shp_file/村1地_全区.shp')
# 这里要控制精度
all_land.geometry = all_land.geometry.apply(shapely.set_precision, grid_size=0.05)
# 对当前 all_land 添加一个 mean_slope 字段 [3.37, 7.38, 6.07, 3.94, 6.53, 7.46, 7.62]
all_land['mean_slope'] = [3.37, 7.38, 6.07, 3.94, 6.53, 7.46, 7.62]

all_land

In [2]:
single_land = CPP_Planner_Kit.get_single_shp(all_land, 6)
# single_land.plot()
split_polygon = CPP_Planner_Kit.split_polygon_by_largest_area(single_land.geometry.iloc[0], tolerance=0.05)
_, ax = plt.subplots()
for polygon in split_polygon:
    ax.plot(*polygon.exterior.xy)

In [3]:
single_polygon = split_polygon[1]
single_polygon_angle = CPP_Planner_Kit.get_land_MABR_angle(single_polygon)
single_polygon = affinity.rotate(single_polygon, -single_polygon_angle, origin='centroid')
mabr = single_polygon.minimum_rotated_rectangle
_, ax = plt.subplots()
ax.plot(*mabr.exterior.xy, color='b')
ax.plot(*single_polygon.exterior.xy, color='y')

In [31]:
corrected_swath_width = CPP_Planner_Kit.get_corrected_swath_width(swath_width=1.45, slope=6)
headland_width, theta = CPP_Planner_Kit.calc_headland_width(turning_radius=4.5, swath_width=corrected_swath_width,
                                                            vehicle_length=6.3
                                                            , vehicle_width=1.9)
print(headland_width)
path, headland = CPP_Algorithms.scanline_algorithm_single_with_headland(
    land=gpd.GeoDataFrame(geometry=[single_polygon]), step_size=corrected_swath_width, along_long_edge=False,
    headland='right', head_land_width=headland_width
)
# 显示
_, ax = plt.subplots()
ax.plot(*single_polygon.exterior.xy, 'b')
path.plot(ax=ax, color='y')
headland.plot(ax=ax, color='gray')
print(headland)

####  开始生成转向路径

In [5]:
path
# 由于是从下往上生成的，因此可以直接从第二根路径开始计算

In [6]:
paths = path.geometry.tolist()
print(type(paths))

In [7]:
from CPP_Planner import CPP_Planner_TurningRail_Maker as maker
# 生成 S 路径模版
basic_S_curve = maker.gen_S_shape_curve(turning_radius=4.5, degree_step=0.2, vehicle_length=6.3, vehicle_width=1.9,
                                        swath_width=corrected_swath_width, buffer=0.3)
basic_S_curve.plot(color='y')

In [8]:
a = paths[0].coords[0]
b = paths[0].coords[1]
[a, b]

In [34]:
turning_curves = []
backward_moves = []
forward_moves = []

# 默认从第二根路径开始计算
for i in range(1, len(paths)):
    temp_path = paths[i]
    pre_path = paths[i-1]
    temp_end_point = temp_path.coords[0]
    # 地头行驶路线
    temp_forward_line = LineString((temp_end_point, (temp_end_point[0] + headland_width, temp_end_point[1])))
    # 转向 S 路线
    # TODO: 将S曲线合并了
    temp_S_line_1 = affinity.translate(basic_S_curve.geometry[0], xoff=temp_forward_line.coords[-1][0], yoff=temp_forward_line.coords[-1][1], zoff=0)
    temp_S_line_2 = affinity.translate(basic_S_curve.geometry[1], xoff=temp_forward_line.coords[-1][0], yoff=temp_forward_line.coords[-1][1], zoff=0)
    # 倒车线
    temp_backward_line = LineString((temp_S_line_2.coords[0], pre_path.coords[1]))
    print(temp_backward_line)
    forward_moves.append(temp_forward_line)
    turning_curves.append(temp_S_line_1)
    turning_curves.append(temp_S_line_2)
    backward_moves.append(temp_backward_line)

# 测试显示
_, ax = plt.subplots()
# ax.plot(*single_polygon.exterior.xy, 'b')
path.plot(ax=ax, color='y', linewidth=1.2)
for line in forward_moves:
    ax.plot(*line.xy, 'orange', linewidth=1)
for line in turning_curves:
    ax.plot(*line.xy, 'b--', linewidth=0.8)
for line in backward_moves:
    ax.plot(*line.xy, 'b:', linewidth=0.8)
ax.set_aspect('equal')

In [10]:
f_m, t_c, b_m = maker.gen_S_turning_paths_in_polygon(path, 0, 4.5, 6.3, 1.9, 1.45, 9, 0.3)
print(type(f_m))

In [ ]:
_, ax = plt.subplots()
ax.plot(*single_polygon.exterior.xy, 'b')
path.plot(ax=ax, color='y', linewidth=0.8)
f_m.plot(ax=ax, color='orange', linewidth=0.8)
t_c.plot(ax=ax, color='g', linewidth=0.8)
b_m.plot(ax=ax, color='black', linewidth=0.8)